In [1]:
# Časť 1: Data Preprocessing

# Importovanie knižníc
import numpy             as np   # NumPy je knižnica Pythonu používaná na prácu s poľami
import matplotlib.pyplot as plt  # Matplotlib je knižnica vizualizácie a grafického vykresľovania údajov pre Python, pyplot sa používa na vykreslenie 2D grafiky 
import pandas            as pd   # Pandas je knižnica Pythonu používaná na manipuláciu a analýzu údajov

In [2]:
# Importovanie training datasetu
dataset_train = pd.read_csv('Google_Stock_Price_Train.csv')   # Načítanie Training datasetu
training_set  = dataset_train.iloc[:, 1:2].values             # Pridelenie setu - všetky row, columns od 1 do 2

In [3]:
dataset_train.head()     # Kontrola výpísania dáť zo súboru (prvých 5)

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"


In [4]:
# Škálovanie setu
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1)) 
training_set_scaled = sc.fit_transform(training_set)

In [5]:
# Vytvorenie dátovej štruktúry s 60 časovými krokmi a s 1 výstupom 
X_train = []
y_train = []
for i in range(60, 1258):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [6]:
# Reshaping - zmena tvaru poľa, bez zmeny údajov
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))